<a href="https://colab.research.google.com/github/LimaCondas/eco-driving-speed-rl/blob/main/My_Gym_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description of Eco-Driving Environment .ipynb File
This file contains a basic implementation of the Eco-Driving gym environment, using Python and the OpenAI Gym library. **The environment is designed to simulate a car driving scenario on a straight, flat road with no air friction or inner cost.**

The car is controlled through acceleration and deacceleration actions, and the traction force is assumed to be fully converted into acceleration of the vehicle. **The reward system considers the traction work and the speed deviation from the safety speed.**

The EcoDrivingEnv class is defined and initialized in the notebook, providing the basic structure for the environment. A test environment is created and run with random actions to verify the environment is working as expected.

A deep learning model is built using the Keras library, which is then used to create an agent to interact with the environment. The keras-RL library is used to train and optimize the agent, using the deep learning model and the EcoDrivingEnv environment.

Overall, this notebook provides a simple but effective starting point for experimenting with Eco-Driving environments and reinforcement learning. This file was inspired by a YouTube video https://youtu.be/bD6V3rcr_54, which provided guidance on implementing the Eco-Driving environment using OpenAI Gym. 

#**0.Install Dependencies**

---



In [ ]:
!pip install gym
!pip install numpy==1.20
!pip install tensorflow==2.5.0
!pip install keras
!pip install keras-rl2
!pip install torch


# **1. Test a Simple Eco-Driving Scenario**


---




> **import packages**

In [ ]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

> **This block is EcoDrivingEnv**

In [ ]:
import gym
from gym import spaces
import numpy as np
import random

class EcoDrivingEnv(gym.Env):
    def __init__(self):
        self.safe_speed_limit = 20 # m/s, equal 72kph
        self.time_step = 0.1 # seconds
        self.mass = 1000 # kg
        self.gravity = 9.81 # m/s^2
        self.MAX_SPEED = 40 # m/s, equal 144kph
        self.traction_coefficient = 0.8
        self.position = 0
        self.speed = 20
        self.work = 0
        self.steps = 0
        self.total_reward = 0
        self.distance = 1000 # m
        self.max_steps = 5000 # 5000 steps, 500s
        self.max_acceleration = 1 # m/s^2

        # Action Space deaccelerate, no action, accelerate
        self.action_space = spaces.Discrete(3) 

        # Observation space [position, speed]
        self.observation_space = spaces.Box(low=np.array([0, 0]), high=np.array([np.inf, np.inf]), dtype=np.float32)

        print("Successfully Initialize EcoDrivingEnv.......")

    def reset(self):
        self.position = 0
        self.speed = 0
        self.work = 0
        self.steps = 0
        self.total_reward = 0
        return np.array([self.position, self.speed], dtype=np.float32)

    def step(self, action):
        # Apply action
        if action == 0:
            acceleration = -self.max_acceleration # -1
        elif action == 1:
            acceleration = 0
        elif action == 2:
            acceleration = self.max_acceleration # +1
        else:
            acceleration = 0


        # Calculate new speed and position
        new_speed = self.speed + acceleration * self.time_step # vt = v0 + a * dt
        new_speed = np.clip(new_speed, 0, self.MAX_SPEED) # speed no bigger than max speed

        # xt = x0 + vt + 0.5 * a * t^2
        new_position = self.position + self.speed * self.time_step + 0.5 * acceleration * self.time_step ** 2

        # Calculate work of traction force
        traction_force = self.traction_coefficient * self.mass * acceleration # F = 0.8 * m * a
        self.work += traction_force * (new_position - self.position) # W = F * dx

        # Calculate speed deviation from safe speed limit
        speed_deviation = abs(new_speed - self.safe_speed_limit)

        # Calculate Reward
        # reward = - 0.0015 * self.work  - 0.9985 * speed_deviation 
        reward = - 0.001 * self.work / (self.work + speed_deviation) - 0.999 * speed_deviation / (self.work + speed_deviation)
        # print('Work: {}\t  speed_deviation:{}'.format(self.work, speed_deviation))

        # Update state
        self.position = new_position
        self.speed = new_speed
        self.steps += 1

        # Check if episode is done, distance or steps satisfied
        done = False
        if self.position >= self.distance or self.steps >= self.max_steps:
          done = True

        # Update info dictionary with additional information
        self.state = np.array([self.position, self.speed], dtype=np.float32)
        info = {'friction_work': self.work, 'speed_deviation': speed_deviation}

        # Return step information
        return self.state, reward, done, info


    def render(self):
        print(f"Position: {self.position:.2f}m, Speed: {self.speed:.2f}m/s, Work: {self.work:.2f}J, Steps: {self.steps}")
        pass

> **Examplify an Eco-Drving Env**

In [ ]:
env = EcoDrivingEnv()

Successfully Initialize EcoDrivingEnv.......


> **Test Environment with Random Action**

In [ ]:
# env.action_space.sample()
# env.observation_space.sample()

In [ ]:
episode = 10

flag = 0
position = []
speed = []

for episode in range(1, episode+1):
  state = env.reset()
  done = False
  score = 0
  
  while not done:
    action = random.choice([0, 1, 2])
    n_state, reward, done, info = env.step(action)
    score += reward

    if not flag:
      position.append(n_state[0])
      speed.append(n_state[1])
  flag = 1

  print('===== Episode:{} Score:{} ====='.format(episode, int(score)))


===== Episode:1 Score:-134 =====
===== Episode:2 Score:-205 =====
===== Episode:3 Score:-158 =====
===== Episode:4 Score:-196 =====
===== Episode:5 Score:-60 =====
===== Episode:6 Score:-145 =====
===== Episode:7 Score:-247 =====
===== Episode:8 Score:-324 =====
===== Episode:9 Score:-195 =====
===== Episode:10 Score:-154 =====


> **SAMPLE to plot the profile of position and speed in 1 episode**

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# position_data = position
# speed_data = speed

# # Create a figure with 2 subplots for position and speed
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
# fig.subplots_adjust(hspace=0.4)

# # Plot position data
# ax1.plot(position_data, '-')
# ax1.set_title('Profile of Position')
# ax1.set_xlabel('Time (step)')
# ax1.set_ylabel('Position')

# # Plot speed data
# ax2.plot(speed_data, '-')
# ax2.set_title('Profile of Speed')
# ax2.set_xlabel('Time (step)')
# ax2.set_ylabel('Speed')

# # Display the plot
# plt.show()


# **2. Create a Deep Learning Model with Keras**


---



In [ ]:
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
states

(2,)

In [ ]:
def build_model(states, actions):
  model = Sequential()
  model.add(Flatten(input_shape=(1,) + states))  # Add a Flatten layer
  model.add(Dense(24, activation='relu', input_shape=states))
  model.add(Dense(24, activation='relu'))
  model.add(Dense(actions, activation='linear'))
  return model

In [ ]:
del model # if error, delete this

In [ ]:
model = build_model(states, actions)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 24)                72        
_________________________________________________________________
dense_13 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 75        
Total params: 747
Trainable params: 747
Non-trainable params: 0
_________________________________________________________________


# **3. Build Agent with keras-RL**


---





In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
  policy = BoltzmannQPolicy()
  memory = SequentialMemory(limit=50000, window_length=1)
  dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                 nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
  return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 99s 10ms/step - reward: -0.0208
6 episodes - episode_reward: -21.067 [-43.640, -9.897] - loss: 0.049 - mae: 4.282 - mean_q: 3.509 - friction_work: 19556.410 - speed_deviation: 13.833

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 90s 9ms/step - reward: -0.0120
4 episodes - episode_reward: -48.040 [-123.526, -18.641] - loss: 0.014 - mae: 2.048 - mean_q: 2.739 - friction_work: 9919.367 - speed_deviation: 15.493

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 88s 9ms/step - reward: -0.0256
4 episodes - episode_reward: -23.121 [-39.249, -9.945] - loss: 0.009 - mae: 1.586 - mean_q: 2.403 - friction_work: 9536.994 - speed_deviation: 15.680

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 91s 9ms/step - reward: -0.0155
4 episodes - episode_reward: -68.065 [-172.956, -14.678] - loss: 0